# 合并数据集：Concat与Append操作
将不同的数据源进行合并是数据科学中最有趣的事情之一，这既包括将两个不同的数据集 非常简单地拼接在一起，也包括用数据库那样的连接（join）与合并（merge）操作处理有 重叠字段的数据集。Series 与DataFrame 都具备这类操作，Pandas 的函数与方法让数据合 并变得快速简单。

先来用pd.concat 函数演示一个Series 与DataFrame 的简单合并操作。之后，我们将介绍Pandas 中更复杂的merge 和join 内存数据合并操作。

首先导入Pandas 和NumPy：

In [1]:
import pandas as pd
import numpy as np

简单起见，定义一个能够创建DataFrame 某种形式的函数，后面将会用到：

In [2]:
def make_df(cols, ind):
    """一个简单的DataFrame"""
    data = {c: [str(c) + str(i) for i in ind] for c in cols}
    return pd.DataFrame(data, ind)

# DataFrame示例
make_df('ABC', range(3))

,A,B,C
0,A0,B0,C0
1,A1,B1,C1
2,A2,B2,C2


## 知识回顾：NumPy数组的合并(Recall: Concatenation of NumPy Arrays)
合并Series 与DataFrame 与合并NumPy 数组基本相同， 后者通过之前介绍的np.concatenate 函数即可完成。你可以用这个函数将两个或两个以上的数组合并成一个数组。

In [3]:
x = [1, 2, 3]
y = [4, 5, 6]
z = [7, 8, 9]
np.concatenate([x, y, z])

array([1, 2, 3, 4, 5, 6, 7, 8, 9])

第一个参数是需要合并的数组列表或元组。还有一个axis 参数可以设置合并的坐标轴方向：

In [4]:
x = [[1, 2],
     [3, 4]]
np.concatenate([x, x], axis=1)

array([[1, 2, 1, 2],
       [3, 4, 3, 4]])

## 通过pd.concat实现简易合并(Simple Concatenation with pd.concat)
Pandas 有一个pd.concat() 函数与np.concatenate 语法类似，但是配置参数更多，功能也更强大：
```python
# Signature in Pandas v0.18
pd.concat(objs, axis=0, join='outer', join_axes=None, ignore_index=False,
          keys=None, levels=None, names=None, verify_integrity=False,
          copy=True)
```

pd.concat() 可以简单地合并一维的Series 或DataFrame 对象，与np.concatenate() 合并数组一样：

In [5]:
ser1 = pd.Series(['A', 'B', 'C'], index=[1, 2, 3])
ser2 = pd.Series(['D', 'E', 'F'], index=[4, 5, 6])
pd.concat([ser1, ser2])

1    A
2    B
3    C
4    D
5    E
6    F
dtype: object

它也可以用来合并高维数据，例如下面的DataFrame：

In [6]:
df1 = make_df('AB', [1, 2])
df2 = make_df('AB', [3, 4])
pd.concat([df1, df2])

,A,B
1,A1,B1
2,A2,B2
3,A3,B3
4,A4,B4


默认情况下，DataFrame 的合并都是逐行进行的（默认设置是axis=0）。与np.concatenate()一样，pd.concat 也可以设置合并坐标轴，例如下面的示例：

In [7]:
df3 = make_df('AB', [0, 1])
df4 = make_df('CD', [0, 1])

In [8]:
pd.concat([df3, df4], axis='columns')

,A,B,C,D
0,A0,B0,C0,D0
1,A1,B1,C1,D1


这里也可以使用axis=1，效果是一样的。但是用axis='columns' 会更直观。

#### 1. 索引重复
np.concatenate 与pd.concat 最主要的差异之一就是Pandas 在合并时会保留索引，即使索引是重复的！例如下面的简单示例：

In [9]:
x = make_df('AB', [0, 1])
y = make_df('AB', [2, 3])
y.index = x.index # 复制索引
pd.concat([x, y])

,A,B
0,A0,B0
1,A1,B1
0,A2,B2
1,A3,B3


In [10]:
pd.concat([x, y]).loc[1]

,A,B
1,A1,B1
1,A3,B3


你会发现结果中的索引是重复的。虽然DataFrame 允许这么做，但结果并不是我们想要的。pd.concat() 提供了一些解决这个问题的方法。

###### (1) 捕捉索引重复的错误。
如果你想要检测pd.concat() 合并的结果中是否出现了重复的索 引，可以设置verify_integrity 参数。将参数设置为True，合并时若有索引重复就会 触发异常。下面的示例可以让我们清晰地捕捉并打印错误信息：

In [11]:
try:
    pd.concat([x, y], verify_integrity=True)
except ValueError as e:
    print("ValueError:", e)

ValueError: Indexes have overlapping values: Int64Index([0, 1], dtype='int64')


###### (2) 忽略索引。
有时索引无关紧要，那么合并时就可以忽略它们，可以通过设置ignore_index 参数来实现。如果将参数设置为True，那么合并时将会创建一个新的整数索引。

In [12]:
pd.concat([x, y], ignore_index=True)

,A,B
0,A0,B0
1,A1,B1
2,A2,B2
3,A3,B3


###### (3) 增加多级索引。
另一种处理索引重复的方法是通过keys 参数为数据源设置多级索引标签，这样结果数据就会带上多级索引：

In [13]:
pd.concat([x, y], keys=['x', 'y'])

A   B
x 0  A0  B0
  1  A1  B1
y 0  A2  B2
  1  A3  B3

示例合并后的结果是多级索引的DataFrame，可以用之前介绍的方法将它转换成我们需要的形式。

#### 2. 类似join的合并
前面介绍的简单示例都有一个共同特点，那就是合并的DataFrame 都是同样的列名。而在实际工作中，需要合并的数据往往带有不同的列名，而pd.concat 提供了一些选项来解决这类合并问题。看下面两个DataFrame，它们的列名部分相同，却又不完全相同：

In [14]:
df5 = make_df('ABC', [1, 2])
df6 = make_df('BCD', [3, 4])
pd.concat([df5, df6])

d:\python37\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  This is separate from the ipykernel package so we can avoid doing imports until


,A,B,C,D
1,A1,B1,C1,NaN
2,A2,B2,C2,NaN
3,NaN,B3,C3,D3
4,NaN,B4,C4,D4


默认情况下，某个位置上缺失的数据会用NaN 表示。如果不想这样，可以用join 和join_ axes 参数设置合并方式。默认的合并方式是对所有输入列进行并集合并（join='outer'）， 当然也可以用join='inner' 实现对输入列的交集合并：

In [15]:
pd.concat([df5, df6], join='inner')

,B,C
1,B1,C1
2,B2,C2
3,B3,C3
4,B4,C4


另一种合并方式是直接确定结果使用的列名，设置join_axes 参数，里面是索引对象构成的列表（是列表的列表）。如下面示例所示，将结果的列名设置为第一个输入的列名：

In [16]:
pd.concat([df5, df6], join_axes=[df5.columns])

,A,B,C
1,A1,B1,C1
2,A2,B2,C2
3,NaN,B3,C3
4,NaN,B4,C4


pd.concat 的合并功能可以满足你在合并两个数据集时的许多需求，操作时请记住这一点。

#### 3. append()方法
因为直接进行数组合并的需求非常普遍，所以Series 和DataFrame 对象都支持append 方 法，让你通过最少的代码实现合并功能。例如，你可以使用df1.append(df2)，效果与 pd.concat([df1, df2]) 一样：

In [17]:
df1.append(df2)

,A,B
1,A1,B1
2,A2,B2
3,A3,B3
4,A4,B4


需要注意的是，与Python 列表中的append() 和extend() 方法不同，Pandas 的append() 不 直接更新原有对象的值，而是为合并后的数据创建一个新对象。因此，它不能被称之为一 个非常高效的解决方案，因为每次合并都需要重新创建索引和数据缓存。总之，如果你需 要进行多个append 操作，还是建议先创建一个DataFrame 列表，然后用concat() 函数一次 性解决所有合并任务。

下一节将介绍另一种功能强大的数据组合方法——类似数据库的数据合并，在pd.merge 里实现。关于concat() 与append() 的更多信息，请参考Pandas 文档中“Merge, Join, and Concatenate”（http://pandas.pydata.org/pandas-docs/stable/merging.html）节。